In [9]:
from whisper.whisper.normalizers import EnglishTextNormalizer
import pandas as pd
import jiwer

In [10]:
normalizer = EnglishTextNormalizer()

In [11]:
# data = pd.read_csv("test_base_dev_other.csv")

original = pd.read_csv("test_base_dev_other.csv")
gpt2 = pd.read_csv("test_gpt2_base_dev_other.csv")

In [14]:
cnt = 0
error_idx = []
for i, row in original.iterrows():
    wer = jiwer.wer(normalizer(row["hypothesis"]), normalizer(row["reference"]))
    if wer == 0:
        error_idx.append(i)

wrong_only = original.drop(error_idx)
wrong_only.reset_index(drop=True)
wrong_only.to_csv("base_dev_wrong.csv", index=False)

In [8]:
cnt = 0
error_sentences = []
for i, row in original.iterrows():
    wer = jiwer.wer(normalizer(row["hypothesis"]), normalizer(row["reference"]))
    error_sentences.append((wer, normalizer(row["hypothesis"]), normalizer(row["reference"]), i))
res = sorted(error_sentences, key=lambda t: t[0], reverse=True)
print(res)

[(1.375, 'speaking of those days of nights addison says', 'speaking of those days or nights edison says years ago one of the great violinists was remenyi', 252), (1.0, '4 5', '45', 110), (1.0, 'sully the aspen with far clay', 'so it has been with barclay', 397), (1.0, 'great', 'bruy', 472), (1.0, 'for maria', 'poor poor maria', 583), (1.0, 'alas proyoric', 'alas poor yorick', 751), (1.0, 'on chains rich on chain', 'aunt jane is rich aunt jane', 1046), (1.0, 'wai wai sita', 'where where is sita', 1395), (0.875, 'deuce is one double one 6 is trebokoy', 'deuces won double and 6s treble coin', 273), (0.8, 'petra you are stupid cenetasha', 'petya you are a stupid said natasha', 615), (0.7777777777777778, 'i will rise sir take a word for it', 'the polite jack replied all right sir take your word for it', 275), (0.75, 'i soon received darned my worsted stocking sweetheart', 'i would sooner see you darning my worsted stockings sweetheart', 146), (0.75, 'from 17 to 20 you have got sortages', 'f

In [38]:
cnt = 0
error_sentences = []
prompt = ""
cnt = 0
for i, row in original.iterrows():
    gpt2_row = gpt2.iloc[i]
    wer = jiwer.wer(normalizer(row["hypothesis"]), normalizer(row["reference"]))
    gpt2_wer = jiwer.wer(normalizer(gpt2_row["hypothesis"]), normalizer(gpt2_row["reference"]))
    if gpt2_wer < wer:
        cnt+=1
        if cnt < 5:
            prompt+=f"Input: {normalizer(row['hypothesis'])} Output: {normalizer(gpt2_row['reference'])}\n"
        if cnt == 8:
            prompt+=f"Input: {normalizer(row['hypothesis'])} Output: {normalizer(gpt2_row['reference'])}\n"
        # print(f"original: {normalizer(row['hypothesis'])}\ngpt2: {gpt2_row['hypothesis']}\nreference: {gpt2_row['reference']}\n\n")
print(prompt)

Input: i wish i tell you something Output: i wish to tell you something
Input: but 1st let us be impersonal be epithet is irreverent blasphemer atheist and infidel are flung at a man not from pity but from envy Output: but 1st let us be impersonal the epithets irreverent blasphemer atheist and infidel are flung at a man not from pity but from envy
Input: and now i will sit down and write a letter to my lord Output: and now i will sit down and write a letter to my lord
Input: said missus pinehaven with a start Output: said missus pine avon with a start
Input: the printing shop founds only a short distance from the devil is tavern and we were invited to visit the establishment Output: the printing shop of field was only a short distance from the devil is tavern and we were invited to visit the establishment

